# Dimensionality Reduction with PCA and LDA


### Table of Contents

* [1. Data](#data)
    * [1.1 Load Data](#load-data)
    * [1.2 EDA](#eda)
* [2. PCA](#pca)
    * [2.1 What is PCA?](#what_is_pca)
    * [2.2 PCA Logic](#pca-logic)
    * [2.3 Applying PCA](#applying-pca)
    * [2.4 PCA Results](#pca-results)
    * [2.5 PCA Analysis](#pca-analysis)
* [3. LDA](#lda)
    * [3.1 What is LDA?](#what-is-lda)
    * [3.2 LDA Logic](#lda-logic)
    * [3.3 Applying LDA](#applying-lda)
    * [3.4 LDA Results](#lda-results)
    * [3.5 LDA Analysis](#lda-analysis)
* [4. Conclusion](#conclusion)
* [5. Full Code](#code)

#### Aims:
1. Apply PCA as a dimensionality reduction technique.
2. Apply Linear Discriminate Analysis (LDA) as a dimensionality reduction technique.




## 1. Data<a class="anchor" id="data"></a>

The dataset that will be used to implement the two dimensionality reduction techniques is the [Stellar Classification Dataset](https://www.kaggle.com/datasets/fedesoriano/stellar-classification-dataset-sdss17) from Kaggle. 
The context of the data, as reported by the dataset, is quoted from Kaggle as
> In astronomy, stellar classification is the classification of stars based on their spectral characteristics. The classification scheme of galaxies, quasars, and stars is one of the most fundamental in astronomy. The early cataloguing of stars and their distribution in the sky has led to the understanding that they make up our own galaxy and, following the distinction that Andromeda was a separate galaxy to our own, numerous galaxies began to be surveyed as more powerful telescopes were built. This datasat aims to classificate stars, galaxies, and quasars based on their spectral characteristics.

The dataset consits of `100,000` observations from the [Sloan Digital Sky Survey](https://www.sdss.org/). Each observation has `17 features` columns and belongs to one of `3 classes` which are either a `star`, `galaxy`, or `quasar`. The goal of the assignment is to use `PCA` and `LDA` to reduce the dimensionality of the dataset. 


### 1.1 Load Data<a class="anchor" id="load-data"></a>

### 1.2 EDA<a class="anchor" id="eda"></a>


## 2. PCA<a class="anchor" id="pca"></a>



### 2.1 What is PCA<a class="anchor" id="what_is_pca"></a>



### 2.2 PCA Logic<a class="anchor" id="pca-logic"></a>


### 2.3 Applying PCA<a class="anchor" id="applying-pca"></a>


### 2.4 PCA Results<a class="anchor" id="pca-results"></a>


### 2.5 PCA Analysis<a class="anchor" id="pca-analysis"></a>

## 3. LDA<a class="anchor" id="lda"></a>


### 3.1 What is LDA<a class="anchor" id="what-is-lda"></a>


### 3.2 LDA Logic<a class="anchor" id="lda-logic"></a>


### 3.3 Applying LDA<a class="anchor" id="applying-lda"></a>


### 3.4 LDA Results<a class="anchor" id="lda-results"></a>


### 3.5 LDA Analysis<a class="anchor" id="lda-analysis"></a>

## 4. Conclusion<a class="anchor" id="conclusion"></a>

## 5. Full Code<a class="anchor" id="code"></a>